In [60]:
from opentrons import protocol_api
from opentrons import simulate
protocol = simulate.get_protocol_api('2.15')

metadata = {'apiLevel': '2.15',
               'protocolName':'ELISA protocol group 4',
    'description': '''Following protocols provided by the Polizzi group, this script purifies centrifuged P.pastoris 1 mL cultures, then performs an ELISA to determine protein of interest concentration. ''',
    'author': 'Group Four'}

def run(protocol:protocol_api.ProtocolContext):
    ########################
    # LOAD LABWARE
    ########################
    
    # LOAD P300 PIPETTE TIPS IN SLOTS 5, 6, 7, 9, 10
    tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 5)
    tiprack_2 = protocol.load_labware('opentrons_96_tiprack_300ul', 6)
    tiprack_3 = protocol.load_labware('opentrons_96_tiprack_300ul', 7)
    tiprack_4 = protocol.load_labware('opentrons_96_tiprack_300ul', 9)
    tiprack_5 = protocol.load_labware('opentrons_96_tiprack_300ul', 10)
    
    # LOAD P300 MULTI-CHANNEL PIPETTE & DEFINE WHICH TIPS TO USE
    p300 = protocol.load_instrument('p300_multi_gen2', 'left', tip_racks=[tiprack_1,tiprack_2,tiprack_3,tiprack_4,tiprack_5])
    # DEFINE PIPETTE FLOW RATE (default is 94, decreased to aid liquid uptake)
    p300.flow_rate.aspirate = 40   
    # DEFINE HOW FAR DOWN PIPETTE GOES (default is 1 mm from bottom of well, increased to 5 to aid uptake & avoid disruption of centrifugation pellet / magnetic beads)
    p300.well_bottom_clearance.aspirate = 5
    p300.well_bottom_clearance.dispense = 5
   
    # LOAD RESERVOIR CONTAINING REAGENTS IN SLOT 1
        #A1 well is 2M NaOH
        #A2 will have over 4 mL Ni-NTA equilibration buffer
        #A3 will have pre aliquoted 200 µL of Pierce NiNTA magnetic agarose beads
        #A4 is Ni-NTA wash buffer 
        #A5 is Ni-NTA elution buffer for beads
        #A6 is ELISA coating solution 
        #A7 is ELISA blocking buffer
        #A8 is PBS-T
        #A9 is ELISA primary antibody
        #A10 is ELISA secondary antibody
        #A11 is PnPP
    reservoir = protocol.load_labware('usascientific_12_reservoir_22ml', 1)
    
    # LOAD ELISA 96 WELL PLATE IN SLOT 3
    plate = protocol.load_labware('thermofischer_96_wellplate_400ul', 3)
    
    # LOAD 96 DEEP WELL PLATE IN SLOT 2
        # samples 1-8 are in col 1, samples 9-16 are in col 2
    deep_well = protocol.load_labware('corning_96_wellplate_360ul_flat', 2)
    
    # LOAD RESERVOIR FOR WASTE LIQUIDS IN SLOT 11
    trash = protocol.load_labware('usascientific_12_reservoir_22ml', 11)

    # LOAD MAGNETIC MODULE WITH ADAPTER AND 96 WELL PLATE IN SLOT 7
    mag_mod = protocol.load_module('magnetic module', 4)
    mag_adapter = mag_mod.load_adapter('opentrons_96_flat_bottom_adapter')
    mag_rack = mag_mod.load_labware('thermofischer_96_wellplate_400ul')

    ########################
    # EXTRACT SUPERNATANTS, CORRECT pH, DUPLICATE SAMPLES
    ########################
    
    # 1 mL CENTRIFUGED CULTURES IN SLOT 2 DEEP WELL PLATE, COLUMNS 1 & 2
    # EXTRACT 800 µL SUPERNATANT INTO NEW COLUMNS 3 & 4
    p300.transfer(800, deep_well.rows()[0][:2], deep_well.rows()[0][2:4])#, mix_after=(3, 50))

    # ADD 80 µL NaOH TO SUPERNATANT SAMPLES
    p300.transfer(80, reservoir["A1"], deep_well.rows()[0][2:4], mix_after=(3, 50))
    
    # PAUSE FOR 1 HOUR FOR SALTS TO SETTLE
    protocol.delay(minutes = 60)
    
    # TAKE TWO 300 µL ALIQUOTS OF NEUTRALISED SAMPLES
    # magnetic plate wells cannot hold 600 µL so divide them up
    # leaves 280 µL of neutralised samples in slot 2, cols 3 & 4
    # samples 1-8 made into two tech reps in cols 5,6
    p300.transfer(300, deep_well.rows()[0][2], deep_well.rows()[0][4:6])#, mix_after=(3, 50))
    #samples 9-16 made into two tech reps in cols 7,8
    p300.transfer(300, deep_well.rows()[0][3], deep_well.rows()[0][6:8])#, mix_after=(3, 50))
    
    ########################
    # Ni-NTA MAGNETIC BEAD PURIFICATION
    ########################
    
    # EQUILIBRATE MAGNETIC BEADS
    # ADD 1800 µL EQUILIBRATION BUFFER TO 200 µL BEAD SLURRY 
    p300.transfer(1800/8, reservoir["A2"], reservoir["A3"], mix_after=(3, 50))
    # MIX THEN TRANSFER TO MAGNETIC PLATE
    p300.transfer(2000/8, reservoir["A3"], mag_rack.rows()[0][0], mix_before=(3, 50))
    # ENGAGE MAGNET
    mag_mod.engage(height_from_base=4.4)
    # PAUSE FOR 2 MINUTES FOR BEADS TO SETTLE
    protocol.delay(minutes = 2)
    # REMOVE EQUILIBRIUM BUFFER
    p300.transfer(1800/8, mag_rack.rows()[0][0], trash["A1"])#, mix_after=(3, 50))
    # DISENGAGE MAGNET
    mag_mod.disengage()       
    
    # EQUILIBRATE BEADS AGAIN
    # ADD 1800 µL EQUILIBRATION BUFFER TO BEADS
    p300.transfer(1920/8, reservoir["A2"], mag_rack.rows()[0][0], mix_after=(3, 50))
    # DISTRIBUTE BEADS INTO FOUR COLUMNS (to accommodate 2 x 16 samples)
    p300.transfer(1920/32, mag_rack.rows()[0][0], mag_rack.rows()[0][1:5], mix_before=(3, 50))
    # ENGAGE MAGNET
    mag_mod.engage(height_from_base=4.4)
    # PAUSE FOR 2 MINUTES FOR BEADS TO SETTLE
    protocol.delay(minutes = 2)
    # REMOVE EQUILIBRIUM BUFFER
    p300.transfer(1800/8, mag_rack.rows()[0][0], trash["A1"])#, mix_after=(3, 50))
    # DISENGAGE MAGNET
    mag_mod.disengage()
    
    # ADD 300 µL SAMPLES TO BEADS
    # (magnet: samples 1-8 duplicates in cols 2 & 3, samples 9-16 duplicates in cols 4 & 5)
    p300.transfer(300, deep_well.rows()[0][4:8], mag_rack.rows()[0][1:5], mix_after=(3, 50))
    # TRANSFER SAMPLE+BEAD MIX BACK TO DEEP WELL PLATE (magnet very strong so this maximises protein binding)
    p300.transfer(300, mag_rack.rows()[0][1:5], deep_well.rows()[0][4:8])
    # PAUSE FOR 5 MINUTES FOR PROTEIN TO BIND BEADS
    protocol.delay(minutes = 5)
    # TRANSFER BACK INTO MAGNETIC PLATE
    p300.transfer(300, deep_well.rows()[0][4:8],mag_rack.rows()[0][1:5])#, mix_after=(3, 50)) 
    # ENGAGE MAGNET
    mag_mod.engage(height_from_base=4.4)
    # PAUSE FOR 3 MINUTES FOR BEADS TO SETTLE
    protocol.delay(minutes = 3)
    # REMOVE 300 µL SUPERNATANT
    p300.transfer(300, mag_rack.rows()[0][1:5], trash["A1"])#, mix_after=(3, 50))
    # DISENGAGE MAGNET
    mag_mod.disengage()  
    
    # ADD 300 µL WASH BUFFER
    p300.transfer(300, reservoir["A4"], mag_rack.rows()[0][1:5], mix_after=(3, 50))
    # ENGAGE MAGNET
    mag_mod.engage(height_from_base=4.4)
    # PAUSE FOR 2 MINUTES FOR BEADS TO SETTLE
    protocol.delay(minutes = 2)
    # REMOVE WASH BUFFER
    p300.transfer(300, mag_rack.rows()[0][1:5], trash["A1"])#, mix_after=(3, 50))
    # DISENGAGE MAGNET
    mag_mod.disengage() 
    
    # ADD 300 µL WASH BUFFER AGAIN
    p300.transfer(300, reservoir["A4"], mag_rack.rows()[0][1:5], mix_after=(3, 50))
    # ENGAGE MAGNET
    mag_mod.engage(height_from_base=4.4)
    # PAUSE FOR 2 MINUTES FOR BEADS TO SETTLE
    protocol.delay(minutes = 2)
    # REMOVE WASH BUFFER
    p300.transfer(300, mag_rack.rows()[0][1:5], trash["A1"])#, mix_after=(3, 50))
    # DISENGAGE MAGNET
    mag_mod.disengage() 
    
    # ADD 100 µL ELUTION BUFFER
    p300.transfer(100, reservoir["A5"], mag_rack.rows()[0][1:5], mix_after=(3, 50))
    # TRANSFER OFF OF MAGNET TO MAXIMISE ELUTION (magnet very strong)
    # samples 1-8 duplicates to deep well plate column 9
    p300.transfer(100, mag_rack.rows()[0][1:3],deep_well.rows()[0][8], mix_after=(3, 50))
    # samples 9-16 duplicates to deep well plate column 10
    p300.transfer(100, mag_rack.rows()[0][3:5],deep_well.rows()[0][9], mix_after=(3, 50))
    # PAUSE FOR 10 MINUTES FOR PROTEIN TO ELUTE
    protocol.delay(minutes = 10)
    # TRANSFER BACK TO MAGNETIC PLATE
    # samples 1-8 to magnetic plate col 6, samples 9-16 to col 7
    p300.transfer(200, deep_well.rows()[0][9:10],mag_rack.rows()[0][5:7], mix_after=(3, 50))
    # ENGAGE MAGNET
    mag_mod.engage(height_from_base=4.4)
    # PAUSE FOR 2 MINUTES FOR BEADS TO SETTLE
    protocol.delay(minutes = 2)
    # TRANSFER 200 µL ELUTED SAMPLE INTO DEEP WELL PLATE COLS 11 & 12
    p300.transfer(200, mag_rack.rows()[0][5], deep_well.rows()[10:12])
    
    ########################
    # ELISA
    ########################
    
    # SET UP DILUTION PLATE - starting with 2x to 32x
    # ADD COATING SOLUTION TO WELLS (cols 1-6, 7-12)
    p300.transfer(100, reservoir["A6"], plate.rows()[0][0:12])
    
    # TRANSFER 100 µL PURIFIED SAMPLES 1-8 TO ELISA PLATE COLUMN 1
    # (100 µL purified samples remain in deep well cols 11 & 12 for reserve)
    p300.transfer(100, deep_well.rows()[0][10], plate.rows()[0][0])#, mix_after=(3, 50))
    # TRANSFER 100 µL PURIFIED SAMPLES 9-16 TO ELISA PLATE COLUMN 7
    p300.transfer(100, deep_well.rows()[0][11], plate.rows()[0][6])#, mix_after=(3, 50))
    
    # MAKING DILUTIONS OF SAMPLES 1-8 FROM COLUMN 1 TO 6
    p300.transfer(100, plate.rows()[0][:5], plate.rows()[0][1:6], mix_after=(3, 50))
    # EMPTY THE EXTRA 100 uL FROM COLUMN 6 so 100 µL in all wells
    p300.transfer(100, plate.rows()[0][5], trash["A1"])#, mix_after=(3, 50))

    # MAKING DILUTIONS OF SAMPLES 9-16 FROM COLUMN 7 TO 12
    p300.transfer(100, plate.rows()[0][6:11], plate.rows()[0][7:12], mix_after=(3, 50))
    # EMPTY THE EXTRA 100 uL FROM COLUMN 12 so 100 µL in all wells
    p300.transfer(100, plate.rows()[0][11], trash["A1"])#, mix_after=(3, 50))
    
    # PAUSE FOR 2 HOURS FOR COATING SOLUTION TO BIND - add cover slip
    protocol.delay(minutes = 120)
    # REMOVE COATING SOLUTION
    p300.transfer(100, plate.rows()[0][12::-1], trash["A1"], change_tip='always')#, mix_after=(3, 50))
    
    # ADD 200 µL BLOCKING BUFFER
    p300.transfer(200, reservoir["A7"], plate.rows()[0][5::-1])
    p300.transfer(200, reservoir["A7"], plate.rows()[0][11:5:-1])
    # PAUSE FOR 1 HOUR FOR BLOCKING SOLUTION TO BIND
    protocol.delay(minutes=60)
    # REMOVE BLOCKING BUFFER
    p300.transfer(200,plate.rows()[0][5::-1], trash["A1"])
    p300.transfer(200,plate.rows()[0][11:5:-1], trash["A1"])
    # WASH 3 TIMES WITH 100 µL PBS-T
    for wash in range(3):
        p300.transfer(100,reservoir["A8"] , plate.rows()[0][5::-1], mix_after=(1, 50))
        p300.transfer(100,reservoir["A8"] , plate.rows()[0][11:5:-1], mix_after=(1, 50))
        p300.transfer(100,plate.rows()[0][5::-1], trash["A1"])
        p300.transfer(100,plate.rows()[0][11:5:-1], trash["A1"])
    
    # ADD 100 µL PRIMARY ANTIBODY
    p300.transfer(100, reservoir["A9"], plate.rows()[0][5::-1])
    p300.transfer(100, reservoir["A9"], plate.rows()[0][11:5:-1])
    # PAUSE FOR 1 HOUR FOR ANTIBODY TO BIND
    protocol.delay(minutes=60)
    # REMOVE ANTIBODY
    p300.transfer(100,plate.rows()[0][5::-1], trash["A2"])
    p300.transfer(100,plate.rows()[0][11:5:-1], trash["A2"])
    # WASH 3 TIMES WITH 100 µL PBS-T
    for wash in range(3):
        p300.transfer(100,reservoir["A8"] , plate.rows()[0][5::-1], mix_after=(1, 50))
        p300.transfer(100,reservoir["A8"] , plate.rows()[0][11:5:-1], mix_after=(1, 50))
        p300.transfer(100,plate.rows()[0][5::-1], trash["A1"])
        p300.transfer(100,plate.rows()[0][11:5:-1], trash["A1"])

    # ADD 100 µL SECONDARY ANTIBODY
    p300.transfer(100, reservoir["A10"], plate.rows()[0][5::-1])
    p300.transfer(100, reservoir["A10"], plate.rows()[0][11:5:-1])
    # PAUSE FOR 1 HOUR FOR ANTIBODY TO BIND
    protocol.delay(minutes=60)
    # REMOVE ANTIBODY
    p300.transfer(100,plate.rows()[0][5::-1], trash["A2"])
    p300.transfer(100,plate.rows()[0][11:5:-1], trash["A2"])
    
    # WASH 3 TIMES WITH 100 µL PBS-T, AGITATING FOR 5 MINUTES
    for wash in range(3):
        p300.transfer(100,reservoir["A8"] , plate.rows()[0][5::-1], mix_after=(5, 50))
        p300.transfer(100,reservoir["A8"] , plate.rows()[0][11:5:-1], mix_after=(5, 50))
        protocol.delay(minutes = 5)
        p300.transfer(100,plate.rows()[0][5::-1], trash["A2"])
        p300.transfer(100,plate.rows()[0][11:5:-1], trash["A2"])

    # ADD 100 µL PnPP, MIX
    p300.transfer(100, reservoir["A11"], plate.rows()[0][5::-1], mix_after=(3, 50))
    p300.transfer(100, reservoir["A11"], plate.rows()[0][11:5:-1], mix_after=(3, 50))
    # PAUSE FOR 15 MINUTES FOR COLOUR TO DEVELOP
    protocol.delay(minutes = 15)
    # ADD 50 µL NaOH TO STOP REACTION, MIX
    p300.transfer(50, reservoir["A1"], plate.rows()[0][5::-1], mix_after=(3, 50))
    p300.transfer(50, reservoir["A1"], plate.rows()[0][11:5:-1], mix_after=(3, 50))

    # MEASURE ABSORBANCE AT 405 nm
    # Absorbance is directly proportional to protein of interest concentration
    
    return


/Users/jonathanfoldi/.opentrons/robot_settings.json not found. Loading defaults
Deck calibration not found.
/Users/jonathanfoldi/.opentrons/deck_calibration.json not found. Loading defaults


In [69]:
from opentrons import protocol_api
from opentrons import simulate
protocol = simulate.get_protocol_api('2.15')

metadata = {'apiLevel': '2.15',
           'protocolName':'ELISA protocol group 4',
'description': '''This protocol is to test the bead cleanup and ELISA for the Polizzi Group''',
'author': 'Group Four'}


tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 5)
tiprack_2 = protocol.load_labware('opentrons_96_tiprack_300ul', 6)
tiprack_3 = protocol.load_labware('opentrons_96_tiprack_300ul', 7)
tiprack_4 = protocol.load_labware('opentrons_96_tiprack_300ul', 9)
tiprack_5 = protocol.load_labware('opentrons_96_tiprack_300ul', 10)

p300 = protocol.load_instrument('p300_multi_gen2', 'left', tip_racks=[tiprack_1,tiprack_2,tiprack_3,tiprack_4,tiprack_5])
p300.flow_rate.aspirate = 40   

p300.well_bottom_clearance.aspirate = 5
p300.well_bottom_clearance.dispense = 5

reservoir = protocol.load_labware('usascientific_12_reservoir_22ml', 1)
plate = protocol.load_labware('thermofischer_96_wellplate_400ul', 3)
deep_well = protocol.load_labware('corning_96_wellplate_360ul_flat', 2)
trash = protocol.load_labware('usascientific_12_reservoir_22ml', 11)

#magnetic module
mag_mod = protocol.load_module('magnetic module', 4)
mag_adapter = mag_mod.load_adapter('opentrons_96_flat_bottom_adapter')
mag_rack = mag_mod.load_labware('thermofischer_96_wellplate_400ul')

#A1 reservoir is 2M NaOH
#A2 will ahve over 4 mL of equilibration buffer
#A3 will have pre allocated 200 uL of beads - then buffer added ot this to make bead slurry
#A4 is the wash buffer for the beads
#A5 is elution buffer for beads
#A6 is coating solution 
#A7 is blocking buffer
#A8 is PBS
#A9 is primary antibody
#A10 is secondary antibody
#A11 is PnPP

#samples 1-8 are in col1 of deep well plate
#samples 9-16 are in col2 of deep well plate

#take 800 uL from deep well plate and put into a new column - cols 3 and 4
p300.transfer(800, deep_well.rows()[0][:2], deep_well.rows()[0][2:4])#, mix_after=(3, 50))

#add 80 uL of NaOH to those columns
p300.transfer(80, reservoir["A1"], deep_well.rows()[0][2:4])#, mix_after=(3, 50))
#delay an hour
#now add 300 uL of each of these to two separate columns because have to split bead cleanup into two
#samples 1-8 made into two tech reps in cols 5,6
p300.transfer(300, deep_well.rows()[0][2], deep_well.rows()[0][4:6])#, mix_after=(3, 50))
#samples 9-16made into two tech reps in cols 7,8
p300.transfer(300, deep_well.rows()[0][3], deep_well.rows()[0][6:8])#, mix_after=(3, 50))


#make the beads 
p300.transfer(1800/8, reservoir["A2"], reservoir["A3"], mix_after=(3, 50))
#transfer beads to mag plate
p300.transfer(2000/8, reservoir["A3"], mag_rack.rows()[0][0], mix_before=(3, 50))
mag_mod.engage(height_from_base=4.4)
#delay two minutes to let the beads get washed
protocol.delay(minutes = 2)
#dump out 1800 uL of this to trash
p300.transfer(1800/8, mag_rack.rows()[0][0], trash["A1"])#, mix_after=(3, 50))
mag_mod.disengage()       
#add new 1800 uL of equil buffer to beads
p300.transfer(1800/8, reservoir["A2"], mag_rack.rows()[0][0], mix_after=(3, 50))

#distribute the mixed beads to four columns of mag plate
p300.transfer(30, mag_rack.rows()[0][0], deep_well.rows()[0][4:8], mix_before=(3, 50),mix_after=(2,25),change_tip = 'always') 
#delay 5 minutes to equilibriate with teh DNA
protoco.delay(minutes = 5)

#transfer beads to mag plate
p300.transfer(330, deep_well.rows()[0][4:8],mag_rack.rows()[0][1:5], mix_after=(3, 50)) 
mag_mod.engage(height_from_base=4.4)
#wait 3 minutes
p300.transfer(660, mag_rack.rows()[0][1:5], trash["A1"], mix_after=(3, 50))
mag_mod.disengage()       
#add wash buffer
p300.transfer(300, reservoir["A4"], mag_rack.rows()[0][1:5], mix_after=(3, 50))
mag_mod.engage(height_from_base=4.4)
#wait 2 mintues
p300.transfer(300, mag_rack.rows()[0][1:5], trash["A1"], mix_after=(3, 50))
mag_mod.disengage()       
p300.transfer(300, reservoir["A4"], mag_rack.rows()[0][1:5], mix_after=(3, 50))
mag_mod.engage(height_from_base=4.4)
#wait 2 mintues
p300.transfer(300, mag_rack.rows()[0][1:5], trash["A1"], mix_after=(3, 50))
mag_mod.disengage()
#elution buffer
p300.transfer(100, reservoir["A5"], mag_rack.rows()[0][1:5], mix_after=(3, 50))

p300.transfer(100, mag_rack.rows()[0][1:3],deep_well.rows()[0][9], mix_after=(3, 50))
p300.transfer(100, mag_rack.rows()[0][3:5],deep_well.rows()[0][10], mix_after=(3, 50))

#wait ten minutes
#add eluted sample abck to mag rack
p300.transfer(200, deep_well.rows()[0][9:10],mag_rack.rows()[0][5:7], mix_after=(3, 50))
#wait two mintues to pull out beads


######### ELISA PROTOCOL ############
#setting up the dilution plate
#adding the antigen solution to col 1
p300.transfer(200, mag_rack.rows()[0][5:7], plate.rows()[0][:1], mix_after=(3, 50))
#adding the coating solution to the other wells for the serial dilutions
p300.transfer(100, reservoir["A6"], plate.rows()[0][1:6])
#making dilutions from col 1 to col 5
p300.transfer(100, plate.rows()[0][:5], plate.rows()[0][1:6], mix_after=(3, 50))
#empty the extra 100 uL from col 6
p300.transfer(100, plate.rows()[0][5], trash["A1"])#, mix_after=(3, 50))

#wait for 2 hours at room temp - at this time, flip the tiprack so that new tips are on the left - now shoudl have 8 cols left
p300.transfer(100, plate.rows()[0][:5], trash["A1"])#, mix_after=(3, 50))
#will need to drop tips after each of these



#add 200 uL blocking buffer to every well
p300.transfer(200, reservoir["A7"], plate.rows()[0][:6])
#wait 1-2 hours at room temp
#dump out blockign buffer
p300.transfer(200,plate.rows()[0][:6], trash["A1"])

#wash with 100 uL of PBS 3 times
for wash in range(3):
    p300.transfer(100,reservoir["A8"] , plate.rows()[0][:6], mix_after=(1, 50))
    p300.transfer(100,plate.rows()[0][:6], trash["A1"])

#chekc how much volume to use per wash

#add 100 uL primary antibody to every well
p300.transfer(100, reservoir["A9"], plate.rows()[0][:6])
#wait 1-2 hours
#dump out solution
p300.transfer(100,plate.rows()[0][:6], trash["A2"])
#wash with 100 uL of PBS 3 times
for wash in range(3):
    p300.transfer(100,reservoir["A8"] , plate.rows()[0][:6], mix_after=(1, 50))
    p300.transfer(100,plate.rows()[0][:6], trash["A2"])


#add 100 uL secondary antibody to every well
p300.transfer(100, reservoir["A10"], plate.rows()[0][:6])
#wait 1-2 hours
#dump out solution
p300.transfer(100,plate.rows()[0][:6], trash["A2"])

#add PBS and agitate for 5 minutes
p300.transfer(100,reservoir["A8"] , plate.rows()[0][:6], mix_after=(1, 50))
###AGITATE FOR 5 MINUTES#####
p300.transfer(100,plate.rows()[0][:6], trash["A2"])
#wash with 100 uL of PBS 3 times
for wash in range(3):
    p300.transfer(100,reservoir["A8"] , plate.rows()[0][:6], mix_after=(1, 50))
    p300.transfer(100,plate.rows()[0][:6], trash["A2"])

#mix the PNPP by hand then add to reservoir column A11
p300.transfer(100, reservoir["A11"], plate.rows()[0][:6], mix_after=(3, 50))
#incubate fro 15- 30 minutes
#add 50 uL of NaOH
p300.transfer(50, reservoir["A1"], plate.rows()[0][:6], mix_after=(3, 50))

for line in protocol.commands():
    print(line)

/Users/jonathanfoldi/.opentrons/robot_settings.json not found. Loading defaults
Deck calibration not found.
/Users/jonathanfoldi/.opentrons/deck_calibration.json not found. Loading defaults
Execution of 49ee993b-d55d-4a4a-b30a-fd886cd8b433 failed
Traceback (most recent call last):
  File "/Users/jonathanfoldi/anaconda3/lib/python3.11/site-packages/opentrons/protocol_engine/state/labware.py", line 344, in get_definition_by_uri
    return self._state.definitions_by_uri[uri]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^
KeyError: 'opentrons/thermofischer_96_wellplate_400ul/1'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/jonathanfoldi/anaconda3/lib/python3.11/site-packages/opentrons/protocol_engine/execution/equipment.py", line 158, in load_labware
    definition = self._state_store.labware.get_definition_by_uri(definition_uri)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File

ProtocolCommandFailedError: Error 4000 GENERAL_ERROR (ProtocolCommandFailedError): PythonException: FileNotFoundError: Labware "thermofischer_96_wellplate_400ul" not found with version 1 in namespace "opentrons".

In [1]:
#custom labware 
A1_x= 14.3
A1_Y = 11.3

Delta_x = 9

Delta_y = 9

Diameter = 9

Well_depth = 10.2


Base_x = 127.7
Base_y = 85.5

Height = 14.4
'''
        "G1": {
            "depth": 11,
            "totalLiquidVolume": 200,
            "shape": "circular",
            "diameter": 7,
            "x": 14.5,
            "y": 20.6,
            "z": 3.4
        },
'''

In [55]:
rows = ['A','B','C','D','E','F','H']
cols = list(range(1,13))

for col_index,col in enumerate(cols):
    for row_index,row in enumerate(rows):
        print('"'+str(row+str(col))+'": {')
        print('    "depth": 10.2,')
        print('    "totalLiquidVolume": 400,')
        print('    "shape": circular,')        
        print('    "diameter": 9,')        
        print('    "x": '+str(14.3+col_index*9)+',')        
        print('    "y": '+str(74.2-row_index*9)+',')        
        print('    "z": 4.5}\n')
        

"A1": {
    "depth": 10.2,
    "totalLiquidVolume": 400,
    "shape": circular,
    "diameter": 9,
    "x": 14.3,
    "y": 74.2,
    "z": 4.5}

"B1": {
    "depth": 10.2,
    "totalLiquidVolume": 400,
    "shape": circular,
    "diameter": 9,
    "x": 14.3,
    "y": 65.2,
    "z": 4.5}

"C1": {
    "depth": 10.2,
    "totalLiquidVolume": 400,
    "shape": circular,
    "diameter": 9,
    "x": 14.3,
    "y": 56.2,
    "z": 4.5}

"D1": {
    "depth": 10.2,
    "totalLiquidVolume": 400,
    "shape": circular,
    "diameter": 9,
    "x": 14.3,
    "y": 47.2,
    "z": 4.5}

"E1": {
    "depth": 10.2,
    "totalLiquidVolume": 400,
    "shape": circular,
    "diameter": 9,
    "x": 14.3,
    "y": 38.2,
    "z": 4.5}

"F1": {
    "depth": 10.2,
    "totalLiquidVolume": 400,
    "shape": circular,
    "diameter": 9,
    "x": 14.3,
    "y": 29.200000000000003,
    "z": 4.5}

"H1": {
    "depth": 10.2,
    "totalLiquidVolume": 400,
    "shape": circular,
    "diameter": 9,
    "x": 14.3,
    "y":

In [51]:
plate.rows()[0][5::-1]

[A6 of Corning 96 Well Plate 360 µL Flat on 2,
 A5 of Corning 96 Well Plate 360 µL Flat on 2,
 A4 of Corning 96 Well Plate 360 µL Flat on 2,
 A3 of Corning 96 Well Plate 360 µL Flat on 2,
 A2 of Corning 96 Well Plate 360 µL Flat on 2,
 A1 of Corning 96 Well Plate 360 µL Flat on 2]